#Introduction:
The 'nltk.logic' module provides support for analyzing expressions of First-Order Logic (FOL).

First-Order Logic (FOL): FOL is a formal system used to represent and reason about statements involving quantifiers (such as “for all” and “there exists”) and predicates (relations between objects). It’s a powerful tool for expressing complex relationships and making logical inferences.
Expression Objects: The nltk.logic package allows you to parse FOL expressions into Expression objects. These objects represent logical statements, including quantified variables, predicates, and logical connectives (such as conjunction, disjunction, implication, and equivalence).

In [7]:
from nltk.sem.logic import *
# the 'fromstring' obj from the 'Expression' module takes string of the terms.
read_expr = Expression.fromstring

In [8]:
# here we created a term, the term is as the following (there exists a man.)
# in FOL: Ex (man(x))
e1 = read_expr('exists x.man(x)')
print(e1)

# the 'alpha_convert' function is used to convert the variable in the term.
# in FOL: Ex (man(x)) ====> Ez (man(z))
e2 = e1.alpha_convert(Variable('z'))
print(e2)

# comparison between the two terms.
# in FOL: is e1 equal e2 ====> is 'Ex (man(x))' == 'Ez (man(z))'
e1 == e2

exists x.man(x)
exists z.man(z)


True

In [9]:
# assigning variables
print(read_expr(r'john'))
print(read_expr(r'x'))
print(read_expr(r'-man(x)'))
print(read_expr(r'--man(x)'))
print(read_expr(r'(man(x))'))

john
x
-man(x)
--man(x)
man(x)


#Predicates and Functions:
Now, what about predicates and functions, we'll be using the 'BinaryExpression()' function, for assigning the functions of the terms, it takes two 'Variables' as parameters. and the 'VariableExpression()' function, for assigning the predicates of the terms.

In [10]:
# here we have the sentence "Sami is tall".
# in FOL: tall(Sami)

# first define Sami as a "Variable"
Sami = read_expr(r'Sami')

# then difine the predicate
tall = read_expr(r'tall(x)')
sami_is_tall = tall(Sami)

sami_is_tall

<ApplicationExpression tall(x,Sami)>

In [11]:
# here we have the sentence "Ali is a teacher".
# in FOL: teacher(Ali)

# we can simply define both the predicate and the function in a single call
ali_is_tall1 = read_expr(r'tall(Ali)')

# or we can write it like this
ali_is_tall2 = read_expr(r'\P.P(Ali)(\x.tall(x))').simplify()

print(ali_is_tall1 == ali_is_tall2)

# but make sure too 'simplify()' the expression
ali_is_tall2

True


<ApplicationExpression tall(Ali)>

In [12]:
# otherwise
ali_is_tall3 = read_expr(r'\P.P(Ali)(\x.tall(x))')
ali_is_tall3

<ApplicationExpression \P.P(Ali)(\x.tall(x))>

In [13]:
ali_is_tall3 == ali_is_tall1

False

In [14]:
# Example: 'Ahmed is taller than Ali'
# in FOL: Taller(Ahmed, Ali)

Ahmed = read_expr(r'Ahmed')
Ali = read_expr(r'Ali')

Taller = read_expr(r'\x.\y.Taller(x, y)')

Taller(Ahmed, Ali)

<ApplicationExpression ((\x y.Taller(x,y))(Ahmed))(Ali)>

In [15]:
Taller(Ahmed, Ali).simplify()

<ApplicationExpression Taller(Ahmed,Ali)>

In [17]:
# this way the expression goes bottom-up
Taller2 = read_expr(r'\P.P(Ahmed)(\x.(\P.P(Ali)(\y.Taller(x,y))))').simplify()
Taller2

<ApplicationExpression Taller(Ahmed,Ali)>

In [18]:
Taller == Taller2

False

In [19]:
Taller.simplify() == Taller2

False

In [20]:
# Taller(x,y) != Taller(Ahmed, Ali)

Taller.simplify(), Taller2.simplify()

(<LambdaExpression \x y.Taller(x,y)>,
 <ApplicationExpression Taller(Ahmed,Ali)>)

In [21]:
Taller(Ahmed, Ali).simplify() == Taller2

True

#Connectives:

predicate logic on itself only expresses facts, and does not provide information on relationships between statements, and this is a crucial step to provide reasoning for the machine in order analyize the knowledge it has, and thats why in first order logic we use connectives between these predicates to provide the relationships between the predicates.

in NLTK we simply use the following characters to represent these connectives:

negation : "-"

conjunction : "&"

disjunction : "|"

implication : "->"

equivalence : "<->"

In [22]:
# Example: Ahmed eats burger and Ali plays basketball
# in FOL: eats(Ahmed, burger) & plays(Ali, basketball)

Ahmed = read_expr(r'Ahmed')
burger = read_expr(r'burger')
eats = read_expr(r'\x.\y.eats(x, y)')
pre1 = eats(Ahmed, burger).simplify()

pre1

<ApplicationExpression eats(Ahmed,burger)>

In [23]:
Ali = read_expr(r'Ali')
basketball = read_expr(r'basketball')
plays = read_expr(r'\x.\y.plays(x, y)')
pre2 = plays(Ali, basketball).simplify()

pre2

<ApplicationExpression plays(Ali,basketball)>

In [24]:
FOL = (pre1 & pre2).simplify()
FOL

<AndExpression (eats(Ahmed,burger) & plays(Ali,basketball))>

In [25]:
# as we learned before, we can assign the term as is:
FOL2 = read_expr(r'eats(Ahmed, burger) & plays(Ali, basketball)').simplify()
FOL == FOL2

True

In [26]:
# other connectives:

john = read_expr('john')
man = read_expr('\\x.man(x)')
walk = read_expr('\\x.walk(x)')

print((man(john) & walk(john)).simplify())
print((man(john) | walk(john)).simplify())
print((man(john) > walk(john)).simplify())
print((man(john) < walk(john)).simplify())

(man(john) & walk(john))
(man(john) | walk(john))
(man(john) -> walk(john))
(man(john) <-> walk(john))


#Quantifiers:
 Quantifiers can be simply aplied by using the the 'exists' keyword for E quantifier(there exists, some etc.) and 'all' for the A quantifier(for all, every etc.)

In [27]:
# Example: 'Some dogs are large'.
# in FOL: Ex (dog(x) & large(x))

dogs = read_expr(r'dogs(x)')
large = read_expr(r'large(x)')
term = (dogs & large).simplify()

term

<AndExpression (dogs(x) & large(x))>

In [28]:
exist = read_expr(r'exists x.P(x)')
exist(term).simplify()

<ApplicationExpression exists x.P(x)((dogs(x) & large(x)))>

In [64]:
# Example: 'There is a tall student in room 105'.
# in FOL: Ex (student(x) & tall(x) & in(x, room 105))
variable = read_expr(r'x')

student = read_expr(r'\x.student(x)')
tall = read_expr(r'\x.tall(x)')
in_func = read_expr(r"\x.in(x, room105)")

exist = read_expr(r'exists x.P')
pre = (student(variable) & tall(variable) & in_func(variable))

FOL = exist(pre).simplify()
print(FOL)

exists x.P((student(x) & tall(x) & in(x,room105)))


In [65]:
# make sure the spacing is correct, only add spaces with connectives and quantifiers
FOL2 = read_expr(r'exists x.P((student(x) & (tall(x) & (in(x,room105)))))').simplify()
print(FOL2)

exists x.P((student(x) & tall(x) & in(x,room105)))
